IMPORT LIBRARIES:

- 'Tokenizer': text tokenization
- 'pad_sequences': sequence padding
- 'to_categorical': converting labels to binary class matrices
- 'LabelEncoder': scikit-learn, encoding text labels as integers

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

2025-09-08 00:16:44.599208: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-08 00:16:44.637157: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-08 00:16:45.542053: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Loading the dataset

In [3]:
data = pd.read_csv('./data/medical_data.csv')
data.head()

,Patient_Problem,Disease,Prescription
0,"Constant fatigue and muscle weakness, struggli...",Chronic Fatigue Syndrome,"Cognitive behavioral therapy, graded exercise ..."
1,"Frequent severe migraines, sensitivity to ligh...",Migraine with Aura,"Prescription triptans, avoid triggers like bri..."
2,"Sudden weight gain and feeling cold, especiall...",Hypothyroidism,Levothyroxine to regulate thyroid hormone levels.
3,"High fever, sore throat, and swollen lymph nod...",Mononucleosis,"Rest and hydration, ibuprofen for pain."
4,"Excessive thirst and frequent urination, dry m...",Diabetes Mellitus,Insulin therapy and lifestyle changes.


Data Preprocessing and Preparation

In [4]:
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(data['Patient_Problem'])

sequences = tokenizer.texts_to_sequences(data['Patient_Problem'])

Padding Sequences

In [5]:
max_length = max(len(x) for x in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

Encoding the Labels and Converting them to Categorical

In [6]:
# Encoding the labels
label_encoder_disease = LabelEncoder()
label_encoder_prescription = LabelEncoder()

disease_labels = label_encoder_disease.fit_transform(data['Disease'])
prescription_labels = label_encoder_prescription.fit_transform(data['Prescription'])

# Converting labels to categorical
disease_labels_categorical = to_categorical(disease_labels)
prescription_labels_categorical = to_categorical(prescription_labels)

Combining Labels into a Multi-label Target Variable

In [7]:
Y = np.hstack((disease_labels_categorical, prescription_labels_categorical))

MODEL BUILDING:

Defining Model Architecture

- Model and Input: define the model architecture
- Embedding: convert the integer sequences into dense vectors of fixed size
- Dense: output layers that make predictions

In [8]:
input_layer = Input(shape=(max_length,))

embedding = Embedding(input_dim=5000, output_dim=64)(input_layer)
lstm_layer = LSTM(64)(embedding)

disease_output = Dense(len(label_encoder_disease.classes_), activation='softmax', name='disease_output')(lstm_layer)

prescription_output = Dense(len(label_encoder_prescription.classes_), activation='softmax', name='prescription_output')(lstm_layer)

I0000 00:00:1757266144.192796   12449 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3864 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Compiling the model

In [9]:
model = Model(inputs=input_layer, outputs=[disease_output, prescription_output])

model.compile(
    loss={'disease_output':'categorical_crossentropy', 'prescription_output':'categorical_crossentropy'},
    optimizer='adam',
    metrics={'disease_output':['accuracy'], 'prescription_output':['accuracy']}
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 17)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 17, 64)    │    320,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 64)        │     33,024 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ disease_output      │ (None, 178)       │     11,570 │ lstm[0][0]        │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prescription_output │ (None, 388)       │     25,220 │ lstm[0][0]        │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 389,814 (1.49 MB)

 Trainable params: 389,814 (1.49 MB)

 Non-trainable params: 0 (0.00 B)

Training the model

In [15]:
# import tensorflow as tf

# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#     print("TensorFlow detected the following GPUs:")
#     for gpu in gpus:
#         print(f"Name: {gpu.name}, Type: {gpu.device_type}")
# else:
#     print("TensorFlow did not detect any GPUs.")

import time
start_time = time.time()

model.fit(padded_sequences,
          {'disease_output': disease_labels_categorical, 'prescription_output': prescription_labels_categorical},
          epochs=1000, batch_size=32)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time:.4f} seconds")

Epoch 1/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - disease_output_accuracy: 1.0000 - disease_output_loss: 0.0043 - loss: 0.0105 - prescription_output_accuracy: 1.0000 - prescription_output_loss: 0.0062
Epoch 2/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - disease_output_accuracy: 1.0000 - disease_output_loss: 0.0043 - loss: 0.0105 - prescription_output_accuracy: 1.0000 - prescription_output_loss: 0.0062
Epoch 3/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - disease_output_accuracy: 1.0000 - disease_output_loss: 0.0042 - loss: 0.0104 - prescription_output_accuracy: 1.0000 - prescription_output_loss: 0.0062
Epoch 4/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - disease_output_accuracy: 1.0000 - disease_output_loss: 0.0042 - loss: 0.0103 - prescription_output_accuracy: 1.0000 - prescription_output_loss: 0.0061
Epoch 5/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - disease_output_accuracy: 1.0000 - disease_output_loss: 0.0042 - loss: 0.0102 - prescription_output_accuracy: 1.0000 - prescrip

Making Predictions

The model is used to make predictions for new patients:

- Pre-processed the patient's symptoms by performing tokenization and padding.
- Feed the pre-processed data into the trained model.
- The model predicts the disease and medication based on the patient's symptoms.
- The predicted disease and medication will be presented.

In [16]:
def make_prediction(patient_problem):
    # Preprocessing the input
    sequence = tokenizer.texts_to_sequences([patient_problem])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post')

    # Making prediction
    prediction = model.predict(padded_sequence)

    # Decoding the predictions
    disease_index = np.argmax(prediction[0], axis=1)[0]
    prescription_index = np.argmax(prediction[1], axis=1)[0]

    disease_predicted = label_encoder_disease.inverse_transform([disease_index])[0]
    prescription_predicted = label_encoder_prescription.inverse_transform([prescription_index])[0]

    print(f"Predicted Disease: {disease_predicted}")
    print(f"Suggested Prescription: {prescription_predicted}")

patient_input = "I've experienced a loss of appetite and don't enjoy food anymore."
make_prediction(patient_input)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Predicted Disease: Depression
Suggested Prescription: Antidepressants; eating nutrient-rich foods.
